In [36]:
# coding=utf-8
import random
import requests
import selenium.webdriver.support.ui as ui
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from custom_conditions import element_has_css_class
from solve_recaptcha import write_stat, check_exists_by_xpath, wait_between, dimention, solve_images
from untils import find_report_link, watch_videos

geckodriver = 'C:\\Users\\Thinh\\Code\\rp-yt-backend\\etc\\geckodriver-v0.21.0-win64\\geckodriver.exe'
binary = 'C:\\Program Files\\Mozilla Firefox\\firefox.exe'
# geckodriver = '/opt/rp-yt-backend/etc/geckodriver-v0.21.0-linux64/geckodriver'
# binary = '/usr/bin/firefox'
api_key = '094c2420f179731334edccbf176dbd79'
PROXY_HOST = '93.155.250.92'
PROXY_PORT = '8080'
capabilities = DesiredCapabilities.FIREFOX.copy()
capabilities['marionette'] = True
capabilities['acceptSslCerts'] = True

In [32]:
def create_browser():
    profile = webdriver.FirefoxProfile()
    profile.set_preference("browser.privatebrowsing.autostart", True)
    profile.set_preference("browser.cache.disk.enable", False)
    profile.set_preference("browser.cache.memory.enable", False)
    profile.set_preference("browser.cache.offline.enable", False)
    profile.set_preference("network.http.use-cache", False)
    profile.set_preference("network.proxy.type", 1)
    profile.set_preference("network.proxy.share_proxy_settings", True)
    profile.set_preference("network.http.use-cache", False)
    profile.set_preference("network.proxy.http", PROXY_HOST)
    profile.set_preference("network.proxy.http_port", int(PROXY_PORT))
    profile.set_preference('network.proxy.ssl', PROXY_HOST)
    profile.set_preference('network.proxy.ssl_port', int(PROXY_PORT))
    profile.set_preference('network.proxy.socks', PROXY_HOST)
    profile.set_preference('network.proxy.socks_port', int(PROXY_PORT))
#     profile.set_preference("general.useragent.override", "whater_useragent")
    options = webdriver.FirefoxOptions()
    # options.add_argument('-headless')
    browser = webdriver.Firefox(
        executable_path=geckodriver,
        firefox_options=options,
        capabilities=capabilities,
        firefox_binary=binary,
        firefox_profile=profile)
    browser.set_window_size(1920, 1080)
    browser.set_window_position(0, 0)
    return browser

# Login

In [13]:
def login(browser, email, password, recovery_mail):
    try:
        browser.get('https://www.youtube.com/')

#         sign_in_btn = browser.find_element_by_css_selector('a.yt-simple-endpoint.style-scope.ytd-button-renderer')
#         sign_in_btn.click()

#         email_inp = browser.find_element_by_id('identifierId')
#         email_inp.send_keys(email)
#         browser.find_element_by_id('identifierNext').click()  # đi đến trang nhập password
#         sleep(2)
#         WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#headingText > content")))
#         heading_text = browser.find_element_by_css_selector('#headingText > content')
#         if heading_text.text == 'Welcome':
#             print(heading_text.text)
#         elif heading_text.text == 'Account disabled':
#             return False

#         password_inp = browser.find_element_by_css_selector('input.whsOnd.zHQkBf')
#         password_inp.send_keys(password)
#         browser.find_element_by_id('passwordNext').click()
#         try:
#             WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.ID, "headingText")))
#             header = browser.find_element_by_id('headingText')
#             if header and header.text == 'Verify it\'s you':
#                 print('Need to input recovery email')
#                 browser.find_element_by_class_name('vdE7Oc').click()
#                 sleep(5)
#                 recovery_email = browser.find_element_by_id('knowledge-preregistered-email-response')
#                 recovery_email.send_keys(recovery_mail)
#                 next_btn = browser.find_element_by_id('next')
#                 next_btn.click()
#         except Exception as ex:
#             print('No need to input recovery email', str(ex))

#         try:
#             # check protected account
#             title = browser.find_element_by_class_name('N4lOwd')
#             if title.text == 'Protect your account':
#                 done_btn = browser.find_element_by_class_name('CwaK9')
#                 done_btn.click()
#         except Exception as ex:
#             print('Can not submit', str(ex))

#         ui.WebDriverWait(browser, 10).until(expected_conditions.url_matches('https://www.youtube.com/'))
#         print('Login success')
        return True
    except Exception as ex:
        print('Login failed', str(ex))
        return False


# Submit report

In [8]:
def submit_report(browser, report_channel, report_reason_1, report_reason_2, report_note):
    try:
        # watch some videos
        browser.get(report_channel + '/videos')
        sleep(2)
        try:
            text = browser.find_element_by_css_selector('#container > yt-formatted-string')
            if text.text == 'This account has been terminated for violating Google\'s Terms of Service.':
                return '2'
        except NoSuchElementException as ex:
            print(ex)

        videos = browser.find_elements_by_id('video-title')
        video = random.choice(videos)
        href = video.get_attribute('href')
        watch_videos(browser, href)

        sleep(random.randint(0, 100))

        for i in range(4):
            browser.execute_script("window.scrollTo(0, {})".format(random.randint(0, 1080)))

        # Report this channel
        report_link = find_report_link(report_channel)
        browser.get(report_link)
        sleep(5)
        tab_start = browser.find_element_by_id('tab-start')
        report_types = tab_start.find_elements_by_css_selector('ul > li > div > label')
        for report_type in report_types:
            if report_type.text == report_reason_1:
                report_type.click()

        tab_hate_speech = browser.find_element_by_id('tab-hate-speech')
        report_types = tab_hate_speech.find_elements_by_css_selector('ul > li > div > label')
        for report_type in report_types:
            if report_type.text == report_reason_2:
                report_type.click()

        continue_btn = browser.find_element_by_id('show-reported-user-info')
        continue_btn.click()

        ui.WebDriverWait(browser, 10).until(
            element_has_css_class((By.TAG_NAME, 'textarea'), "yt-uix-form-input-textarea"))
        notes = browser.find_element_by_class_name('yt-uix-form-input-textarea')
        notes.send_keys(report_note)

        html = browser.find_element_by_tag_name('html')
        html.send_keys(Keys.END)
        return '1'
    except Exception as ex:
        print('Submit report failed', str(ex))
        return '0'

# Goto recaptcha

## get src using in 2captcha api

In [9]:
def get_key_recaptcha(browser):
    browser.switch_to.default_content()
    iframe_switch = browser.find_element(By.XPATH, "/html/body/div[1]/div[3]/div/div/form/div[2]/div/div[19]/div[4]/div/div/div/iframe")
    key = iframe_switch.get_attribute('src')
    keys = key.split('&')
    for item in keys:
        if 'k=' in item:
            key = item[2:]
    return key

In [10]:
def key_resolver_captcha(api_key, api_url):
    r = requests.get(api_url)
    res = r.text
    if 'OK' in res:
        request_id = res[3:]
        resolver_api = 'http://2captcha.com/res.php?key={}&action=get&id={}'.format(api_key, request_id)
        print(resolver_api)
        while True:
            response = requests.get(resolver_api)
            response = response.text
            if 'OK' in response:
                response_key = response[3:]
                sleep(5)
                break
            if 'ERROR_CAPTCHA_UNSOLVABLE' in response:
                response_key = None
                break

        return response_key
    else:
        print('Can not get key api 2captcha.com')


In [11]:
def change_language(browser):
    lang_btn = browser.find_element_by_id('yt-picker-language-button')
    lang_btn.click()
    vi_btn = browser.find_elements_by_css_selector('div.yt-picker-grid:nth-child(5) > button:nth-child(2)')
    vi_btn.click()


## Main

In [38]:
email = 'pam.beaulieu.velez@gmail.com'
password = 'Minh1234'
recovery_email = 'graynice64108@yahoo.com'
browser = create_browser()
login_status = login(browser, email, password, recovery_email)

## get key

In [37]:
api_key = '094c2420f179731334edccbf176dbd79'
googlekey = get_key_recaptcha()
current_url = browser.current_url
captcharesolver_api = 'http://2captcha.com/in.php?key={}&method=userrecaptcha&googlekey={}&pageurl={}&here=now'.format(api_key, googlekey, current_url)
key_resolver = key_resolver_captcha(api_key, captcharesolver_api)

TypeError: get_key_recaptcha() missing 1 required positional argument: 'browser'

## resolver captcha after receiver request

In [53]:
# browser.switch_to.default_content()
# browser.delete_all_cookies()
# browser.switch_to.frame(iframeSwitch)
#ActionChains(driver).move_to_element(iframeSwitch).perform()
# browser.delete_all_cookies()
# WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.ID, "recaptcha-anchor")))
# ele = browser.find_element(By.ID, "recaptcha-anchor")
#ActionChains(driver).move_to_element(ele).perform()
# ele.click()


In [54]:
browser.switch_to.default_content()
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.ID, "g-recaptcha-response")))
browser.execute_script("document.getElementById('g-recaptcha-response').style.display = 'block';")
textarea_box = browser.find_element_by_id('g-recaptcha-response')
textarea_box.send_keys(key_resolver)

In [55]:
submit_report = browser.find_element_by_id('submit-report')
submit_report.click()

In [56]:
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.ID, "content")))
section = browser.find_element_by_css_selector('.section > p:nth-child(1)')
if section.text == 'Thank You.':
    sleep(5)
    browser.quit()

# check if channel die

In [ ]:
text = browser.find_element_by_css_selector('#container > yt-formatted-string')
if text.text == 'This account has been terminated for violating Google\'s Terms of Service.':
    print('True')
